# MONA Lisa


a) Write the static method numberOfLeapYears, which returns the number of leap years between
year1 and year2, inclusive.
In order to calculate this value, a helper method is provided for you.
• isLeapYear(year) returns true if year is a leap year and false otherwise.
Complete method numberOfLeapYears below. You must use isLeapYear appropriately to
receive full credit.
/** Returns the number of leap years between year1 and year2, inclusive.
 * Precondition: 0 <= year1 <= year2
 */
 public static int numberOfLeapYears(int year1, int year2)

In [ ]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String asciiFile; 
    private String greyScaleFile;
    private String redScaleFile;
    private String blueScaleFile;
    private String greenScaleFile;
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;

    // Constructor obtains attributes of picture
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.resizedFile = this.outDir + name + "." + ext;
        this.asciiFile = this.outDir + name + ".txt";
        this.greyScaleFile = this.outDir + "Grey" + name + "." + ext;
        this.redScaleFile = this.outDir + "Red" + name + "." + ext;
        this.blueScaleFile = this.outDir + "Blue" + name + "." + ext;
        this.greenScaleFile = this.outDir + "Green" + name + "." + ext;
        this.setStats();
    }

    
    // An image contains metadata, namely size, width, and height
    public void setStats() {
        BufferedImage img;
        try {
            Path path = Paths.get(this.inFile);
            this.bytes = Files.size(path);
            img = ImageIO.read(new File(this.inFile));
            this.width = img.getWidth();
            this.height = img.getHeight();
        } catch (IOException e) {
        }
    }

    // Console print of data
    public void printStats(String msg) {
        System.out.println(msg + ": " + this.bytes + " " + this.width + "x" + this.height + "  " + this.inFile);
    }

    // Convert scaled image into buffered image
    public static BufferedImage convertToBufferedImage(Image img) {

        // Create a buffered image with transparency
        BufferedImage bi = new BufferedImage(
                img.getWidth(null), img.getHeight(null),
                BufferedImage.TYPE_INT_ARGB);

        // magic?
        Graphics2D graphics2D = bi.createGraphics();
        graphics2D.drawImage(img, 0, 0, null);
        graphics2D.dispose();

        return bi;
    }
    
    // Scale or reduce to "scale" percentage provided
    public void resize(int scale) {
        BufferedImage img = null;
        Image resizedImg = null;  

        int width = (int) (this.width * (scale/100.0) + 0.5);
        int height = (int) (this.height * (scale/100.0) + 0.5);

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            // create a new BufferedImage for drawing
            resizedImg = img.getScaledInstance(width, height, Image.SCALE_SMOOTH);
        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(resizedImg), this.ext, new File(resizedFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.resizedFile;  // use scaled file vs original file in Class
        this.setStats();
    }
    
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void convertToAscii() {
        BufferedImage img = null;
        PrintWriter asciiPrt = null;
        FileWriter asciiWrt = null;

        try {
            File file = new File(this.asciiFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        try {
            asciiPrt = new PrintWriter(asciiWrt = new FileWriter(this.asciiFile, true));
        } catch (IOException e) {
            System.out.println("ASCII out file create error: " + e);
        }

        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }

        for (int i = 0; i < img.getHeight(); i+=3) {
            for (int j = 0; j < img.getWidth(); j+=1) {
                Color col = new Color(img.getRGB(j, i));
                double pixVal = (((col.getRed() * 0.30) + (col.getBlue() * 0.59) + (col
                        .getGreen() * 0.11)));
                try {
                    asciiPrt.print(asciiChar(pixVal));
                    asciiPrt.flush();
                    asciiWrt.flush();
                } catch (Exception ex) {
                }
            }
            try {
                asciiPrt.println("");
                asciiPrt.flush();
                asciiWrt.flush();
            } catch (Exception ex) {
            }
        }
    }

    // conversion table, there may be better out there ie https://www.billmongan.com/Ursinus-CS173-Fall2020/Labs/ASCIIArt
    public String asciiChar(double g) {
        String str = " ";
        if (g >= 240) {
            str = " ";
        } else if (g >= 210) {
            str = ".";
        } else if (g >= 190) {
            str = "*";
        } else if (g >= 170) {
            str = "+";
        } else if (g >= 120) {
            str = "^";
        } else if (g >= 110) {
            str = "&";
        } else if (g >= 80) {
            str = "8";
        } else if (g >= 60) {
            str = "#";
        } else {
            str = "@";
        }
        return str;
    }

    public void greyScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.greyScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int grey = (int)(col.getRed()*0.3 + col.getGreen()*0.59 + col.getBlue()*0.11);
                try {
                    Color rgb = new Color(grey, grey, grey);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.greyScaleFile));
        } catch (Exception ex) {
        }
    }

    public void redScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.redScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int red = (int)(col.getRed());
                try {
                    Color rgb = new Color(red, 0, 0);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.redScaleFile));
        } catch (Exception ex) {
        }
    }

    public void greenScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.greenScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int green = (int)(col.getGreen());
                try {
                    Color rgb = new Color(0, green, 0);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.greenScaleFile));
        } catch (Exception ex) {
        }
    }

    public void blueScale() {
        BufferedImage img = null;
        try {
            File file = new File(this.blueScaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }
        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                int blue = (int)(col.getBlue());
                try {
                    Color rgb = new Color(0, 0, blue);
                    img.setRGB(j, i, rgb.getRGB());
                } catch (Exception ex) {
                }
            }
        }
        try {
            ImageIO.write(img, "png", new File(this.blueScaleFile));
        } catch (Exception ex) {
        }
    }

    // tester/driver
    public static void main(String[] args) throws IOException {
        Pics ascii = new Pics("MonaLisa", "png");
        ascii.printStats("Original");
        ascii.resize(33);
        ascii.printStats("Scaled");
        ascii.convertToAscii();

        Pics grey = new Pics("MonaLisa", "png");
        grey.greyScale();

        Pics red = new Pics("MonaLisa", "png");
        red.redScale();
        
        Pics green = new Pics("MonaLisa", "png");
        green.greenScale();

        Pics blue = new Pics("MonaLisa", "png");
        blue.blueScale();
    }
}
Pics.main(null);

The Method "numberOfLeapYears" outputs the number of leap years that there are between a certain set of years. The helper method "isLeapYear(y)" is the main part of this function and returns true given an input of a leap year and false given the input of a non-leap year.

1b)
Write the static method dayOfWeek, which returns the integer value representing the day of the week for
the given date (month, day, year), where 0 denotes Sunday, 1 denotes Monday, ..., and 6
denotes Saturday. For example, 2019 began on a Tuesday, and January 5 is the fifth day of 2019. As a result,
January 5, 2019, fell on a Saturday, and the method call dayOfWeek(1, 5, 2019)
returns 6.
As another example, January 10 is the tenth day of 2019. As a result, January 10, 2019, fell on a Thursday,
and the method call dayOfWeek(1, 10, 2019) returns 4.
In order to calculate this value, two helper methods are provided for you.
• firstDayOfYear(year) returns the integer value representing the day of the week for the first
day of year, where 0 denotes Sunday, 1 denotes Monday, ..., and 6 denotes Saturday. For
example, since 2019 began on a Tuesday, firstDayOfYear(2019) returns 2.
• dayOfYear(month, day, year) returns n, where month, day, and year specify the
nth day of the year. For the first day of the year, January 1 (month = 1, day = 1), the value
1 is returned. This method accounts for whether year is a leap year. For example,
dayOfYear(3, 1, 2017) returns 60, since 2017 is not a leap year, while
dayOfYear(3, 1, 2016) returns 61, since 2016 is a leap year. 

In [ ]:
public static int dayOfWeek(int month, int day, int year) {

    int firstDay = firstDayOfYear(year);

    int day = dayOfYear(month, day, year);

    //subtract one because the calendar doesn't start at 0
    return (firstDay + day - 1) % 7;
}

The method firstDayOfYear outputs a value that shows what day of the week a date on the calendar is. Sunday is represented by a value of zero, Monday by one, up until Saturday which is represented by six.

The methoddayOfYear tells us what day a certain calendar date on. For example, dayOfYear( 7, 6, 2011 ) would return 6 because the 6th of January is the sixth day of the year in 2011.